# Seq2Seq Code Generation — Google Colab Training
### Trains 3 models: Vanilla RNN | LSTM | Attention LSTM
**Before running:** Go to `Runtime → Change runtime type → GPU (T4)`

## Step 1 — Upload Your Project Files
Upload all `.py` files and `config_quick.yaml` from your local machine to Colab.

In [ ]:
from google.colab import files
import os

# Upload all project files at once
# Select these files from your computer:
# train.py, evaluate.py, visualize_attention.py, generate_report_plots.py,
# data_loader.py, run.py, config_quick.yaml, requirements.txt
# models/ folder files: __init__.py, vanilla_rnn.py, lstm.py, attention_lstm.py

uploaded = files.upload()
print(f"Uploaded: {list(uploaded.keys())}")

In [ ]:
# Create models/ directory and upload model files separately
os.makedirs('models', exist_ok=True)
os.chdir('models')
print("Now upload the 4 model files: __init__.py, vanilla_rnn.py, lstm.py, attention_lstm.py")
model_files = files.upload()
os.chdir('..')
print("Model files uploaded:", list(model_files.keys()))
print("Current directory files:", os.listdir('.'))

## Step 2 — Install Dependencies

In [ ]:
!pip install datasets sacrebleu seaborn pyyaml tqdm -q
print("All dependencies installed!")

## Step 3 — Check GPU
If you see "No GPU", go to `Runtime → Change runtime type → T4 GPU` and re-run.

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("Training will be FAST on GPU!")
else:
    print("No GPU found! Go to Runtime -> Change runtime type -> GPU")
    print("Training will be very slow on CPU.")

## Step 4 — Set Config for Colab
Writes the config file with 10k training examples, 5 epochs, GPU enabled.

In [ ]:
colab_config = """
dataset:
  name: "Nan-Do/code-search-net-python"
  train_size: 10000
  val_size: 1000
  test_size: 1000
  max_docstring_length: 50
  max_code_length: 80
  cache_dir: "./data_cache"

model:
  embedding_dim: 256
  hidden_dim: 256
  dropout: 0.3
  max_vocab_size: 10000

training:
  batch_size: 64
  num_epochs: 5
  learning_rate: 0.001
  teacher_forcing_ratio: 0.5
  gradient_clip: 5.0
  save_every: 1

paths:
  checkpoints: "./checkpoints"
  results: "./results"
  visualizations: "./visualizations"
  logs: "./logs"

device: "cuda"
"""

with open('config_colab.yaml', 'w') as f:
    f.write(colab_config.strip())

print("config_colab.yaml created!")
print("Settings: 10k examples | 5 epochs | batch=64 | embedding=256 | hidden=256")

## Step 5 — Train All 3 Models
This will take approximately **30–60 minutes** on a T4 GPU.
Progress bars will show loss for each epoch.

In [ ]:
!python train.py --config config_colab.yaml --model all

## Step 6 — Evaluate All Models
Calculates BLEU score, Token Accuracy, and Exact Match on the test set.

In [ ]:
!python evaluate.py --config config_colab.yaml --model all --split test

## Step 7 — Generate Attention Heatmaps & Report Plots

In [ ]:
# Generate attention heatmaps (5 examples + summary stats)
!python visualize_attention.py --config config_colab.yaml --num_examples 5 --summary

# Generate report plots (training curves, model comparison, performance by length)
!python generate_report_plots.py --config config_colab.yaml

# Show the attention heatmaps inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

viz_dir = './visualizations/attention'
png_files = sorted([f for f in os.listdir(viz_dir) if f.endswith('.png')])

print(f"\nFound {len(png_files)} visualization files:")
for f in png_files:
    print(f"  {f}")

# Display first 5 attention heatmaps
fig, axes = plt.subplots(1, min(5, len(png_files)), figsize=(25, 6))
if len(png_files) == 1:
    axes = [axes]

for i, fname in enumerate(png_files[:5]):
    img = mpimg.imread(os.path.join(viz_dir, fname))
    axes[i].imshow(img)
    axes[i].set_title(fname.replace('.png', ''), fontsize=9)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Step 8 — Download Everything
Downloads a single zip file containing: trained models, evaluation results, and all visualizations.

In [ ]:
import zipfile
import os
from google.colab import files

def zip_folder(folder_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
        for root, dirs, filenames in os.walk(folder_path):
            for filename in filenames:
                filepath = os.path.join(root, filename)
                zf.write(filepath)
    print(f"Zipped: {zip_name} ({os.path.getsize(zip_name)/1e6:.1f} MB)")

# Zip checkpoints (trained model weights)
zip_folder('./checkpoints', 'checkpoints.zip')

# Zip results (BLEU scores, evaluation JSON files, plots)
zip_folder('./results', 'results.zip')

# Zip visualizations (attention heatmaps)
zip_folder('./visualizations', 'visualizations.zip')

# Download all 3 zips to your computer
print("\nDownloading files...")
files.download('checkpoints.zip')
files.download('results.zip')
files.download('visualizations.zip')
print("Done! Check your Downloads folder.")